In [175]:
import numpy as np
import pandas as pd

有序的索引和无序的索引-前面有提过索引切片问题的时候提到索引排序，这里细说

__如果 MultiIndex 不是有序的索引，那么大多数切片操作都会失败__

In [176]:
index = pd.MultiIndex.from_product([['a','c','b'],[1,2],['x','y','z']],names=['char','int','xyz'])
data = pd.Series(np.random.rand(18),index=index)
type(data)

pandas.core.series.Series

In [177]:
data

char  int  xyz
a     1    x      0.393543
           y      0.024658
           z      0.925126
      2    x      0.067376
           y      0.820321
           z      0.262419
c     1    x      0.142827
           y      0.785813
           z      0.646108
      2    x      0.262706
           y      0.536357
           z      0.721422
b     1    x      0.077714
           y      0.328592
           z      0.707198
      2    x      0.997268
           y      0.822463
           z      0.079609
dtype: float64

对索引排序

``` data['a':'c'] # UnsortedIndexError ```

In [178]:
data.index.is_lexsorted()

False

In [179]:
data2 = data.sort_index()

In [180]:
data2 # 注意索引顺序

char  int  xyz
a     1    x      0.393543
           y      0.024658
           z      0.925126
      2    x      0.067376
           y      0.820321
           z      0.262419
b     1    x      0.077714
           y      0.328592
           z      0.707198
      2    x      0.997268
           y      0.822463
           z      0.079609
c     1    x      0.142827
           y      0.785813
           z      0.646108
      2    x      0.262706
           y      0.536357
           z      0.721422
dtype: float64

In [181]:
data2.index.is_lexsorted()

True

In [182]:
data2['a':'b']

char  int  xyz
a     1    x      0.393543
           y      0.024658
           z      0.925126
      2    x      0.067376
           y      0.820321
           z      0.262419
b     1    x      0.077714
           y      0.328592
           z      0.707198
      2    x      0.997268
           y      0.822463
           z      0.079609
dtype: float64

索引stack与unstack:行列转化

1、unstack：可以通过 level 参数设置转换的索引层级 level=-1默认:最后一个

2、stack：unstack的逆向操作，但是有点不同

3、他们的操作都是放入最内维度

In [183]:
data2.unstack(level=0)   # 可以通过 level 参 数设置转换的索引层级 level=-1默认:最后一个，整数位索引的层级

char            a         b         c
int xyz                              
1   x    0.393543  0.077714  0.142827
    y    0.024658  0.328592  0.785813
    z    0.925126  0.707198  0.646108
2   x    0.067376  0.997268  0.262706
    y    0.820321  0.822463  0.536357
    z    0.262419  0.079609  0.721422

In [184]:
data2.unstack(level=0).index # 可以看到索引变了

MultiIndex(levels=[[1, 2], ['x', 'y', 'z']],
           labels=[[0, 0, 0, 1, 1, 1], [0, 1, 2, 0, 1, 2]],
           names=['int', 'xyz'])

In [185]:
data2.unstack(level=0).columns # 可以看到，abc变为columns

Index(['a', 'b', 'c'], dtype='object', name='char')

In [186]:
type(data2.unstack(level=0))  # 可以看到，这已经不是一个Series  ,而是一个DataFrame

pandas.core.frame.DataFrame

In [187]:
data2.unstack(level=0).unstack(level=0)

char         a                   b                   c          
int          1         2         1         2         1         2
xyz                                                             
x     0.393543  0.067376  0.077714  0.997268  0.142827  0.262706
y     0.024658  0.820321  0.328592  0.822463  0.785813  0.536357
z     0.925126  0.262419  0.707198  0.079609  0.646108  0.721422

In [188]:
data2.unstack(level=0).unstack(level=0).unstack(level=0)

char  int  xyz
a     1    x      0.393543
           y      0.024658
           z      0.925126
      2    x      0.067376
           y      0.820321
           z      0.262419
b     1    x      0.077714
           y      0.328592
           z      0.707198
      2    x      0.997268
           y      0.822463
           z      0.079609
c     1    x      0.142827
           y      0.785813
           z      0.646108
      2    x      0.262706
           y      0.536357
           z      0.721422
dtype: float64

In [189]:
data2.unstack(level=0).unstack(level=0).unstack(level=0).index

MultiIndex(levels=[['a', 'b', 'c'], [1, 2], ['x', 'y', 'z']],
           labels=[[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2], [0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1], [0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]],
           names=['char', 'int', 'xyz'])

从上面的结果可以看出，不会把所有的index都转化为columns，可能是因为没有意义吧

In [190]:
data3 = data2.unstack().unstack()

In [191]:
data3

xyz          x                   y                   z          
int          1         2         1         2         1         2
char                                                            
a     0.393543  0.067376  0.024658  0.820321  0.925126  0.262419
b     0.077714  0.997268  0.328592  0.822463  0.707198  0.079609
c     0.142827  0.262706  0.785813  0.536357  0.646108  0.721422

In [192]:
data3.stack(level=0) #  stack也一样，是unstack的反向操作，level 也一样，默认-1

int              1         2
char xyz                    
a    x    0.393543  0.067376
     y    0.024658  0.820321
     z    0.925126  0.262419
b    x    0.077714  0.997268
     y    0.328592  0.822463
     z    0.707198  0.079609
c    x    0.142827  0.262706
     y    0.785813  0.536357
     z    0.646108  0.721422

In [193]:
data3.stack(level=0).stack(level=0)  # 但是这里不同，他会把所有的columns转到index中，
                                     #并且继续转化会报错，因为Series、DataFrame不能没有索引

char  xyz  int
a     x    1      0.393543
           2      0.067376
      y    1      0.024658
           2      0.820321
      z    1      0.925126
           2      0.262419
b     x    1      0.077714
           2      0.997268
      y    1      0.328592
           2      0.822463
      z    1      0.707198
           2      0.079609
c     x    1      0.142827
           2      0.262706
      y    1      0.785813
           2      0.536357
      z    1      0.646108
           2      0.721422
dtype: float64

索引的设置与重置

层级数据维度转换的另一种方法是行列标签转换，可以通过 reset_index 方法实现

In [198]:
data2

char  int  xyz
a     1    x      0.393543
           y      0.024658
           z      0.925126
      2    x      0.067376
           y      0.820321
           z      0.262419
b     1    x      0.077714
           y      0.328592
           z      0.707198
      2    x      0.997268
           y      0.822463
           z      0.079609
c     1    x      0.142827
           y      0.785813
           z      0.646108
      2    x      0.262706
           y      0.536357
           z      0.721422
dtype: float64

In [199]:
pop_flat = data2.reset_index(name='value')

In [200]:
pop_flat  # 这是一般数据的基本样式

,char,int,xyz,value
0,a,1,x,0.393543
1,a,1,y,0.024658
2,a,1,z,0.925126
3,a,2,x,0.067376
4,a,2,y,0.820321
5,a,2,z,0.262419
6,b,1,x,0.077714
7,b,1,y,0.328592
8,b,1,z,0.707198
9,b,2,x,0.997268


In [202]:
pop_flat.set_index(['char','int','xyz'])  # 转化成 DataFrame

value
char int xyz          
a    1   x    0.393543
         y    0.024658
         z    0.925126
     2   x    0.067376
         y    0.820321
         z    0.262419
b    1   x    0.077714
         y    0.328592
         z    0.707198
     2   x    0.997268
         y    0.822463
         z    0.079609
c    1   x    0.142827
         y    0.785813
         z    0.646108
     2   x    0.262706
         y    0.536357
         z    0.721422